In [1]:
import numpy as np
import pandas as pd
import argparse
import os
import math

In [2]:
DATA2D = {'adult': 'target',
          'compas': 'ScoreText_',
          'german': 'loan_status',
          'synthetic' : 'D'}

DATA2S = {'adult': 'sex',
          'compas': 'Ethnic_Code_Text_',
          'german': 'sex',
          'synthetic': 'S'}

NAMES = ['adult', 'compas', 'german', 'synthetic']

In [5]:
name = "german"
num_X = None
fold = 1
use_fair = True
threshold = 0.5

# Synthetic support WIP
if name == "synthetic":
    filename = '../data/synthetic_data/%s.csv' % num_X
    train_splits = '../data/splited_data/10-cv/synthetic/%s/train_ids.csv' % num_X
    test_splits = '../data/splited_data/10-cv/synthetic/%s/test_ids.csv' % num_X
else:
    assert(name in NAMES)
    filename = '../data/processed_data/%s_binerized.csv' % name
    labels_filename = '../exp/%s/1/para/max-ll/predict-per-example-proxy-label.csv' % name
    train_splits = '../data/splited_data/10-cv/%s/train_ids.csv' % name
    test_splits = '../data/splited_data/10-cv/%s/test_ids.csv' % name

train_id = np.array(pd.read_csv(train_splits)['x%s' % fold]) - 1
test_id = np.array(pd.read_csv(test_splits)['x%s' % fold]) - 1

train_id = train_id[~np.isnan(train_id)].astype(int)
test_id = test_id[~np.isnan(test_id)].astype(int)

cloumns = [DATA2D[name], 'Df'] if name == "synthetic" else [DATA2D[name]]
decision_label = 'Df' if (use_fair and name =="synthetic") else DATA2D[name]

train_data = pd.read_csv(filename).iloc[train_id, :]
test_data = pd.read_csv(filename).iloc[test_id, :]
labels = pd.read_csv(labels_filename)
pdf_train_x = labels['P(Df|e) train_x']
pdf_test_x = labels['P(Df|e) test_x']
binarized_pdf_train_x = pdf_train_x >= threshold
binarized_pdf_test_x = pdf_test_x >= threshold
train_sex = train_data["sex"]
test_sex = test_data["sex"]

    # return train_data, test_data, cloumns, decision_label, train_y_fair, train_y_proxy, test_y_fair, test_y_proxy

In [6]:
cloumns

['loan_status']

In [7]:
cloumns.append(DATA2S["german"])
x_train = np.array(train_data.drop(columns=cloumns))
x_test = np.array(test_data.drop(columns=cloumns))

In [8]:
from sklearn.linear_model import LogisticRegression
from fairlearn.reductions import ExponentiatedGradient
from fairlearn.reductions import DemographicParity

In [9]:
learn = ExponentiatedGradient(
        LogisticRegression(solver='liblinear', fit_intercept=True),
        constraints=DemographicParity())

learn.fit(x_train, binarized_pdf_train_x, sensitive_features=train_sex)


ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x177962410>,
                      estimator=LogisticRegression(solver='liblinear'),
                      nu=0.006982320521359312)

In [12]:
prob_test = learn._pmf_predict(x_test)[:, 1]
prob_train = learn._pmf_predict(x_train)[:, 1]

In [14]:
# Metrics
from fairlearn.metrics import (
    MetricFrame, plot_model_comparison,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

In [13]:
def get_metrics_df(models_dict, y_true, group):
    metrics_dict = {
        "Overall selection rate": (
            lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        "Overall balanced error rate": (
            lambda x: 1-balanced_accuracy_score(y_true, x), True),
        "Balanced error rate difference": (
            lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        " ------": (lambda x: "", True),
        "False positive rate difference": (
            lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
        "False negative rate difference": (
            lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "AUC difference": (
            lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
        df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores) 
                                for model_name, (preds, scores) in models_dict.items()]
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

(810,)

In [ ]:
models_dict = {"Unmitigated": (prob_test, test_scores)}
get_metrics_df(models_dict, Y_test, A_str_test)